In [74]:
import pandas as pd
import numpy as np
from tqdm import trange

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import torch.nn as nn

In [75]:
df = pd.read_csv('C:/Python Projects/MachineLearning/AirBnB/listings.csv', index_col=0)
df['price'] = df['price'].replace('[\$,]', '', regex=True)
df = df.dropna(subset=['price'])
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df = df[df['price'] < 1000]
df = df.reset_index()
print(df.describe)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\luigi\AppData\Local\Temp\ipykernel_3948\3137394758.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True)


<bound method NDFrame.describe of                         id                                       listing_url  \
0                     6499                 https://www.airbnb.com/rooms/6499   
1                    25659                https://www.airbnb.com/rooms/25659   
2                    29396                https://www.airbnb.com/rooms/29396   
3                    29915                https://www.airbnb.com/rooms/29915   
4                    33348                https://www.airbnb.com/rooms/33348   
...                    ...                                               ...   
20875  1371005127220718285  https://www.airbnb.com/rooms/1371005127220718285   
20876  1371091268401865721  https://www.airbnb.com/rooms/1371091268401865721   
20877  1371216308686882761  https://www.airbnb.com/rooms/1371216308686882761   
20878  1371223704329486155  https://www.airbnb.com/rooms/1371223704329486155   
20879  1371344702478038562  https://www.airbnb.com/rooms/1371344702478038562   

     

In [76]:
scalar_df = df[["price", "review_scores_rating", "review_scores_value", "number_of_reviews", "beds"]]
location_data = df[['latitude', 'longitude', 'price']]

In [77]:
#one-hot-encoding
import ast

data = df

# Convert amenities column from string to list
data['amenities'] = data['amenities'].apply(ast.literal_eval)

print(type(data['amenities'].iloc[0])) 

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(data['amenities'])

X = pd.DataFrame(one_hot, columns=mlb.classes_)

print(X.head())
print(X['Dishes and silverware'])

amenity_counts = X.sum().sort_values(ascending=False)
print(amenity_counts)
# Get top 250 amenities by count
top_250_amenities = amenity_counts.head(250).index.tolist()

# Reduce X to only these amenities
X_reduced = X[top_250_amenities]
X_reduced.shape
X = X_reduced

print(X)

<class 'list'>
    Balay stainless steel oven   Scandinavian White body soap  \
0                            0                              0   
1                            0                              0   
2                            0                              0   
3                            0                              0   
4                            0                              0   

    Scandinavian White shampoo   mitsubishi  refrigerator  \
0                            0                          0   
1                            0                          0   
2                            0                          0   
3                            0                          0   
4                            0                          0   

   ( First Day amenities provided) body soap  \
0                                          0   
1                                          0   
2                                          0   
3                                  

In [78]:
amenity_data = X
amenity_data['price'] = df['price']
print(amenity_data.head)

<bound method NDFrame.head of        Kitchen  Wifi  Hot water  Hair dryer  Essentials  \
0            1     1          1           0           1   
1            1     1          1           1           1   
2            1     1          1           1           1   
3            1     1          1           1           1   
4            1     1          0           0           0   
...        ...   ...        ...         ...         ...   
20875        1     1          1           1           1   
20876        1     1          1           1           1   
20877        1     1          0           0           0   
20878        0     0          0           0           0   
20879        1     1          1           1           0   

       Dishes and silverware  Hangers  Iron  Refrigerator  Cooking basics  \
0                          1        0     0             0               1   
1                          1        1     0             1               1   
2                          1  

In [79]:
nans_to_remove = amenity_data[amenity_data.isnull().any(axis=1)].index

In [80]:
print(scalar_df.describe())
print(location_data.describe())
print(amenity_data['price'])
print(np.mean(amenity_data['price']))

              price  review_scores_rating  review_scores_value  \
count  20880.000000          18208.000000         18207.000000   
mean     114.203736              4.660640             4.574730   
std      103.830370              0.407197             0.420694   
min        8.000000              1.000000             1.000000   
25%       60.000000              4.560000             4.470000   
50%       87.000000              4.760000             4.670000   
75%      130.000000              4.900000             4.810000   
max      990.000000              5.000000             5.000000   

       number_of_reviews          beds  
count       20880.000000  20843.000000  
mean           73.676580      2.396152  
std           110.176745      1.907590  
min             0.000000      0.000000  
25%             4.000000      1.000000  
50%            27.000000      2.000000  
75%            95.000000      3.000000  
max          1718.000000     35.000000  
           latitude     longitude   

In [81]:
location_data = location_data.drop(index=nans_to_remove)
scalar_data = scalar_df.drop(index=nans_to_remove)
amenity_data = amenity_data.drop(index=nans_to_remove)

In [82]:
location_data = location_data.reset_index()
scalar_data = scalar_data.reset_index()
amenity_data = amenity_data.reset_index()

In [91]:
import torch
class AirbnbNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

scalar_model = AirbnbNN()
state_dict = torch.load('C:/Python Projects/MachineLearning/final_project_Hermjohannes_Zhang_vonLeitner/final_project_Hermjohannes_Zhang_vonLeitner/models/scalar_model.pth')
scalar_model.load_state_dict(state_dict)

<All keys matched successfully>

In [92]:
import joblib
location_model = joblib.load('C:/Python Projects/MachineLearning/final_project_Hermjohannes_Zhang_vonLeitner/final_project_Hermjohannes_Zhang_vonLeitner/models/XG_boosted_tree_model_lower_price_dataset.pkl')
amenity_model = joblib.load('C:/Python Projects/MachineLearning/final_project_Hermjohannes_Zhang_vonLeitner/final_project_Hermjohannes_Zhang_vonLeitner/models/random_forest_amenities_model_lower_price_dataset.pkl')

In [94]:
y = scalar_data['price']
X_scalar = scalar_data[['review_scores_rating', "review_scores_value", "number_of_reviews", 'beds']]
X_location = location_data[['latitude', 'longitude']]
X_amenities = amenity_data.drop('price', axis=1)

scalar_model.eval()
with torch.no_grad():
    scalar_preds = scalar_model(torch.tensor(X_scalar.to_numpy(), dtype=torch.float32))

location_preds = location_model.predict(X_location)
amenity_preds = amenity_model.predict(X_amenities)

meta_data = pd.DataFrame([scalar_preds, location_preds, amenity_preds], columns=['scalar_preds, location_preds, amenity_preds'])

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 40 inch HDTV with standard cable
- Bosch refrigerator
- Lousani body soap
- Sound system with Bluetooth and aux
- index
Feature names seen at fit time, yet now missing:
- Baby monitor
- Clothing storage: walk-in closet, closet, and dresser
- Free driveway parking on premises
- Stainless steel double oven


In [95]:
print(amenity_data.dtypes)

index                                    int64
Kitchen                                  int64
Wifi                                     int64
Hot water                                int64
Hair dryer                               int64
                                        ...   
40 inch HDTV with standard cable         int64
Bosch refrigerator                       int64
Lousani body soap                        int64
Sound system with Bluetooth and aux      int64
price                                  float64
Length: 252, dtype: object
